In [1]:
import pandas as pd
import numpy as np
import math
import random

from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data_utils
import torchvision.transforms as T
from torch.autograd import Variable

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import shutil
import os
import gc
import gym
import gym_test

In [2]:
env = gym.make('test-v0')

32975653
Discrete(49)
Discrete(32975653)
32975653
49


In [3]:
actions = []
rewards = []
count = 5

env.reset()
while True:
  action = env.action_space.sample()
  reward = env.step(action)
  actions.append(action)
  rewards.append(reward)
  count -= 1
  if count==0:
    break

In [4]:
env.df_xy.head()

,dt,chid,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,...,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,shop_tag
0,1,10321418,3,3891.965283,3,0,0,0,1.00,0.00,...,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1,45
1,1,10414574,2,10616.561549,2,0,0,0,1.00,0.00,...,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1,15
2,1,10134567,2,23527.655416,0,2,0,0,0.00,1.00,...,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1,48
3,1,10001003,9,17751.558260,7,2,0,0,0.75,0.25,...,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1,48
4,1,10267183,1,21701.307598,1,0,0,0,1.00,0.00,...,6.0,15.0,1.0,99.0,30.0,0.000000,0.0,6.0,0,2


In [5]:
BUFFER_SIZE = int(1e5)  #replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate
UPDATE_EVERY = 4        # how often to update the network
EPSILON = 0.8           # probability of chosing on-policy action

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [6]:
sc = StandardScaler()
X_train, X_test, Y_train, Y_test = train_test_split(sc.fit_transform(env.df_xy.iloc[:, :-1].values), env.df_xy.iloc[:, -1].values, test_size=0.1, random_state=1)

X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()

train_loader = data_utils.DataLoader(data_utils.TensorDataset(X_train, Y_train), batch_size=BATCH_SIZE, shuffle=True)

D:\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:1015: RuntimeWarning: overflow encountered in square
  temp **= 2
D:\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:1021: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction ** 2 / new_sample_count
D:\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:1021: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction ** 2 / new_sample_count
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:80: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


In [7]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(52, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        self.fc5 = nn.Linear(24, 49)
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))
        return x

In [8]:
class Agent():
  def __init__(self, action_size, seed):
    self.action_size = action_size
    self.seed = random.seed(seed)


    # Q - Network
    self.qnet_local = DQN().double().to(device)
    self.qnet_target = DQN().double().to(device)

    self.optimizer = optim.Adam(self.qnet_local.parameters(), lr=0.001)

    self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

    self.t_step = 0
    self.train_loss = []

  def step(self, state, action, reward, next_state, done):
    self.memory.add(state, action, reward, next_state, done)


    # learn every 4 timesteps
    self.t_step = (self.t_step+1)%64
    if self.t_step == 0:
      experience = self.memory.sample()
      #print('Experience sampled from memory : ', experience)
      self.learn(experience, GAMMA)


  def epsilon_greedy_action(self, state):
    state = state.to(device)
    self.qnet_local.eval()
    with torch.no_grad():
      action_values = self.qnet_local(state).max(1)[1]#.view(1, 1)
    self.qnet_local.train()

    if random.random() < 0.8:
      print('Predicted action based on QNetwork : ', action_values)
      return action_values.cpu()
    else:
      random_action = random.choices(np.arange(self.action_size), k=BATCH_SIZE)
      print('Chosing  random actions for the batch : ', random_action)
      return torch.DoubleTensor(random_action)
  
  def learn(self, experiences, gamma):
    #print('Started learning')
    states, actions, rewards, next_states, done = experiences#experiences[0].state, experiences[0].action, experiences[0].reward, experiences[0].next_state, experiences[0].done 
    criterion = torch.nn.BCELoss()
    self.qnet_local.train()
    self.qnet_target.eval()

    #predicted_targets = self.qnet_local(states)#.gather(1, actions)

    #print(next_states.view(1, 1))
    with torch.no_grad():
      labels_next = self.qnet_target(next_states).detach().max(1)[0].unsqueeze(1)
    
    #print('labels_next {}'.format(labels_next))
    
    labels = 0 + (gamma * labels_next)
    predicted_targets = self.qnet_local(states).gather(1, actions.long())

    #print("Predicted targets : {}, labels : {}".format(predicted_targets, labels))

    loss = criterion(predicted_targets, labels).to(device)
    print("===========================Training loss ============================")
    print(loss.item())
    self.train_loss.append(loss.item())
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    #print('Total training losses : ', self.train_loss)

    #perform soft update
    self.soft_update(self.qnet_local, self.qnet_target, TAU)
  
  def soft_update(self, local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau*local_param.data + (1-tau)*target_param.data)

In [9]:
class ReplayBuffer():
  def __init__(self, action_size, buffer_size, batch_size, seed):
    self.action_size = action_size
    self.memory = deque(maxlen=buffer_size)
    self.batch_size = batch_size
    self.experiences = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    self.seed = random.seed(seed)
  
  def add(self, state, action, reward, next_state, done):
    experience = self.experiences(state, action, reward, next_state, done)
    self.memory.append(experience)

  def sample(self):
     experiences = random.sample(self.memory, k=self.batch_size)
     #print('Experiences : ', experiences)
     states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).double().to(device)
     actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).double().to(device)
     rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).double().to(device)
     next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).double().to(device)
     dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None])).double().to(device)
     return (states, actions, rewards, next_states, dones)

    # experiences = random.sample(self.memory, k=BATCH_SIZE)

    # batch = self.experiences(*zip(experiences))

    # states = torch.cat(batch.state)
    # actions = torch.cat(batch.actions)
    # rewards = torch.cat(batch.reward)
    # next_states = torch.cat(batch.next_state)
    # dones = torch.cat(batch.done)
    #return random.sample(self.memory, BATCH_SIZE)

    
  
  def __len__(self):
      return len(self.memory)

In [ ]:
# check reward strategy once
# add probability to epsilon_greedy
import json
deep_agent = Agent(action_size=49, seed=0)
num_episodes = 1
max_t = 1000
state = 0
env.state_idx = 0

true_positive = []
true_negative = []

false_positive = []
false_negative = []

TPR = []
FPR = []

current_state = env.df_xy.iloc[0, :-1].values

for i in range(1):  
  #print("==========================EPOCH {} COMPLETED===================".format(i))  

  print('Current state : ', i)
  score = 0
  for state_idx, data in enumerate(train_loader, 0):
    inputs, labels = data    
    action = deep_agent.epsilon_greedy_action(inputs)
    for a in action:
      #print('action taken : ', action)
      next_state, reward, done, info = env.step(a)
      next_state = next_state.iloc[:-1].values
      #print('Next state dtype ; ', next_state.dtype)
      #print('Next state ', next_state)
      deep_agent.step(current_state, a, reward, next_state, done)
      current_state = next_state
      #state = next_state
      #score += reward
      #roc_info = json.loads(info)
      